# Commands and Variables

In [ ]:
#We will implement the method in the arXiv:1707.07658v2 paper

import math
import cmath
import numpy as np
import io
import matplotlib
#matplotlib.use("Agg")
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.path as path
import matplotlib.animation as animation
from IPython.display import HTML


from projectq import MainEngine  # import the main compiler engine
from projectq.ops import H, Swap, Tensor, Toffoli, QFT, All, X, Z, Measure, CNOT, StatePreparation, Rx, Ry, Rz  # import the operations we want to perform (Hadamard and measurement)
from projectq.meta import Control, Dagger, Loop
from projectq.backends import CircuitDrawer
from projectq.types._qubit import Qureg

from openfermion.ops import FermionOperator, QubitOperator  #the creation annihilation operators
from openfermion.utils import hermitian_conjugated
from openfermion.transforms import jordan_wigner, get_sparse_operator
from openfermion.utils import eigenspectrum, trotterize_exp_qubop_to_qasm as trot, error_operator

circuit_backend = CircuitDrawer()
#eng = MainEngine(circuit_backend) #uncomment if we want to get a circuit drawing
eng = MainEngine(engine_list=[])
#eng = MainEngine()

N_lattice = 3
precision_eigenvalue = 1
precision_evolution_time = 0.1
systemsizeA = 1

In [ ]:
import time
from contextlib import contextmanager
from timeit import default_timer

@contextmanager
def elapsed_timer():
    start = default_timer()
    elapser = lambda: default_timer() - start
    yield lambda: elapser()
    end = default_timer()
    elapser = lambda: end-start

# General Functions 


# Hamiltonian setup

In this section we will define the Hamiltonian $Q^\dagger = \sum_{i=0}^{i=N} (1-c_{i-1}^\dagger c_{i-1}) c_i^\dagger (1-c_{i+1}^\dagger c_{i+1})$, where $c_{-1}=c_{N}=0$

In [ ]:
def CountingOperator(
        i
):  #This creates cdagger c at some position i and takes the boundary conditions into account

    if i == -1:  #Then we are at the first position so there is no left neighbour
        return FermionOperator()  #We return '0' so (1 - n_i) = 1
    if i == N_lattice:  #Then we are at the final lattice point and we have no right noughbour
        return FermionOperator()
    i_string_dagger = str(i) + '^ '  #This creates 'i^ '
    i_string = str(i)  #This creates 'i'
    total_string = i_string_dagger + i_string  #'i^ i'
    c_dagger_c = FermionOperator(total_string)  #This makes cdagger_i c_i

    return c_dagger_c

def Q_dagger_term(i):  #This creates all the individual terms of the sum
    i_dagger = str(i) + '^ '  #'i^ '
    Identity = FermionOperator('')
    term = (Identity - CountingOperator(i - 1)) * FermionOperator(i_dagger) * (
        Identity - CountingOperator(i + 1))  #This is the term we will sum

    return term

Q_dagger = FermionOperator()  #Create empty operator

for i in range(N_lattice):  #Now we create our Operator Q
    Q_dagger += Q_dagger_term(i)

Q = hermitian_conjugated(
    Q_dagger)  #We take the Hermitian Conjugate of this operator

Hamiltonian = Q * Q_dagger + Q_dagger * Q  #This will be our Hamiltonian

# Trotterization setup

We take the Jordan-Wigner transformation of the Hamiltonian. Then we use the openfermion command to generate the trotterized version of this Hamiltonian. This gives us the operator $e^{-iH \delta t}$, for some timestep $\delta t$. This operator applied to a state time evolves the state under the Hamiltonian.

In [ ]:
Hamiltonian_jw = jordan_wigner(
    Hamiltonian)  #We take the jordan-wigner transformation

#print(Hamiltonian_jw)

#This outputs a generator of the code in the QASM language that have to be applied after eachother on the quantum computer
Hamiltonian_trot = list(trot(
    Hamiltonian_jw, evolution_time=precision_evolution_time))

#NOTE THAT ttrot returns a generator that can only be used once! So we would need to do this trotteriztion very often which would be computationally intensive so we must save it.
# def Hamiltonian_trot_to_list(Hamiltonian_trot):
#     Hamiltonian = ''
#     for line in Hamiltonian_trot:
#         Hamiltonian += str(line)
#         Hamiltonian += '\n'
#     return Hamiltonian

#Since openfermion generates code in the QASM format I need to change it a bit to get code in the ProjectQ format.
#This command takes as input the lines created by the generator and performs the operations on the qubits in projectq
def QASM_to_ProjectQ(string, qubits):
    #eng.flush()
    if string[:4] == 'CNOT':  #we perform a CNOT
        position_1 = int(string[5])
        position_2 = int(string[7])
        #print('time for CNOT of ', State_to_String(qubits))        
        CNOT | (qubits[position_1], qubits[position_2])
        #print('CNOT done')

    if string[:1] == 'H':  #If we read out a hadamard we perform itWe perform the Hadamard
        #print('time for Hadamard', State_to_String(qubits))
        H | qubits[int(string[2])]
        #print('Hadamard done')

    if string[:1] == 'R':  #We perform a rotation in x,y or z
        #we check where the second ' ' is and take the numbers until there as the angle
        axes = string[1]
        Operation = string[:2]
        angle = float(string[3:string.find(' ', 3)])
        #print('time for Rotation in '+axes+'direction', State_to_String(qubits))
        eval(Operation)(angle) | qubits[int(string[-1])]
        #print('Rotation done')
    eng.flush()

#This command time evolves some state with the Hamiltonian for some evolution time
def time_evolve(qubits, Hamiltonian_trot):
#     Hamiltonian_trot_QASM = io.StringIO(Hamiltonian_trot_to_list(Hamiltonian_trot))
    for line in Hamiltonian_trot:
        QASM_to_ProjectQ(line, qubits)   

In [ ]:
sop = get_sparse_operator(Hamiltonian_jw)
sop.todense()

# Initial State Preparation and desired Operator

This function makes from the all zero state the state $\frac{1}{\sqrt{3}}(|01\rangle + |10 \rangle + |11\rangle$). With this function we make the operator that adds a minus to our state of interest and leaves all other states untouched, thus it sends $|\Phi \rangle \to |\Phi \rangle$ except for our state $|\Psi \rangle \to -|\Psi \rangle$.

In [ ]:
def Generate_3_Bell_States_Superposition(q2):
    #Bring the state from |00> -> 1/sqrt3 |00> + sqrt2/sqrt3 |10>, the factor of two comes in from the fact that rotations around Y have the angle divided by 2.
    Ry(2.0 * np.arccos(3**(-0.5))) | q2[0]

    #Now we do a controlled Hadamard to go to 1/sqrt3 (|00> + |10> + |11> )
    with Control(eng, q2[0]):
        H | q2[1]

    #Now one would want to bring the |00> state to |01> this can be done as follows
    """
    X | q2[0]
    with Control(eng, q2[0]):
        X | q2[1]
    X | q2[0]
    """

    #But in this case only an X gate on the second qubit would already work
    #1/sqrt3 (|00> + |10> + |11> ) -> 1/sqrt3( |01> + |11> + |10> )
    X | q2[1]


#with this function one can just setup the desired state more easily by just stating the desired states.
#Note that the state |001>, that intuitively corresponds to the value |1> in decimal state is an X gate on the first qubit (i.e. X | q[0]).
#And the state |100>, intuitively corresponds to the value |4> in the decimal state and is an x gate on the third qubit (i.e. X | q[2]).
def String_State_Tuple(state_string):
    """As the input string we want e.g. '0.5|0010> + 0.5|0011> + 0.5**0.5 |1000>' this will be converted 
    to the decimal representation '0.5 |2> + 0.5|3>  0.5**0.5 |8>' which will then be converted into the list
    [0, 0, 0.5, 0.5, 0, 0, 0, 0, 0.5**0.5, 0, 0, 0, 0, 0, 0, 0]"""

    #We add a space before and after every state
    state_string = state_string.replace('|', ' |')
    state_string = state_string.replace('>', '> ')

    #We look for the brackets in the string
    state_tuple_list = []
    split_string = state_string.split()

    for i in range(len(split_string)):  #split up the string
        elem = split_string[i]
        elem_previous = split_string[i - 1]
        pos1 = elem.find('|')  #search for a state
        if (pos1 == -1):  #we did not find a state in this string
            continue
        pos2 = elem.find('>')
        qubit_value = 0
        if (pos1 != -1 and pos2 == (len(elem) - 1)
            ):  #i.e. we found a '|' somewhere and '>' is on the last position
            qubit_string = elem[(pos1 + 1):pos2]
            for i in range(len(qubit_string)):
                if (qubit_string[-i - 1] == '1'):
                    qubit_value += 2**i
            #there are two possibilities on the input, either there was a space before or not
            #if there a space we have the amplitude in the element in front of the state_string
            if (pos1 == 0):
                try:
                    qubit_amplitude = eval(elem_previous)
                except SyntaxError:  #then we don't have a wave number in front and the wave number is 1
                    if (
                            len(split_string) > 1
                    ):  #if there is more than one element the wavefunction is not normalized
                        print('Error Wave function is not properly defined')
                        return
                    qubit_amplitude = 1.0
            #if there was no space the amplitude is in front of the ket
            else:
                qubit_amplitude = eval(elem[:pos1])
        state_tuple = (qubit_value, qubit_amplitude)
        state_tuple_list.append(state_tuple)

    return (state_tuple_list)


def Check_normalization(state_list):
    assert type(state_list) is list  #check if argument is a list

    probability_sum = 0
    for i in range(len(state_list)):
        assert type(state_list[i]) is tuple  #check if argument is a tuple
        probability_sum += state_list[i][1]**2

    #check difference with one
    diff = abs(probability_sum - 1.0)

    if diff < 10**-5:
        return True
    else:
        return False


def String_Tuple_ToState(tuple_string, qubits):
    #Here we create a list with acculaed like [0,0,0,0.5,0.5,0.5,0] etc. for the state preparation
    assert type(tuple_string) is list
    #assert type(qubits) is Qureg

    #check if the qubits are in the zero state
    all_zero_string = ''
    for i in range(
            len(qubits)):  #generate list with 0's of length of the qureg
        all_zero_string += '0'

    #Get probability for all zero state
    eng.flush()
    prob_allzero = eng.backend.get_probability(
        all_zero_string,
        qubits)  #also does not work since we have multiple qubits
    assert round(prob_allzero,7) == 1.0

    accu_list = []

    for i in range(2**len(qubits)):
        for k in range(len(tuple_string)):
            if (tuple_string[k][0] == i):
                accu_list.append(tuple_string[k][1])
                break
        else:
            accu_list.append(0)

    assert np.linalg.norm(accu_list) == 1.0  #check if list is normalized
    assert len(accu_list) == 2**len(
        qubits)  #Check if we did not append anything extra by accident

    #return accu_list
    #Now we perform the state preparation
    eng.flush()
    StatePreparation(accu_list) | qubits

    #Now we have the numbers of the states. Then we make a list of size 2^(number of qubits) and use the statepreparation algorithm


def String_to_State(state_string, qubits):
    String_Tuple_ToState(String_State_Tuple(state_string), qubits)


def State_to_String(qubits):
    '''In this function we will return a string with the state from the regular qubit input to help visualizing the state.'''
    state_string = ''

    assert type(qubits) is Qureg

    no_qubits = len(qubits)  #number of qubits

    #We create the list with the wavefunction
    wave_round = [
    ]  #list with rounded values of the coefficients of the qubits

    #We will use the get_amplitude backend function, for this we need to input the desired state
    for i in range(2**no_qubits):
        value_bin = bin(i)  #returns '0b10101'
        value_bin = value_bin[2:]  #Gets rid of '0b'
        while (len(value_bin) < no_qubits):
            value_bin = '0' + value_bin
        assert len(value_bin) == no_qubits
        #because the backend.get functions see the binary numbers inverted i.e. '0001' is 8 instead of our '1000' is 8 we invert the string
        eng.flush()
        probability = eng.backend.get_probability(
            value_bin[::-1], qubits
        )  #get_amplitude only works on calling all qubits, so this scheme does not work
        prob_round = np.round(probability, 8)
        wave_round.append(prob_round)

    #print(wave_round)

    #We loop over the entries of the wave function
    for i in range(len(wave_round)):
        if (wave_round[i] > 0.0):
            #We bring the state value in decimal to a binary value
            value_bin = bin(i)  #returns '0b10101'
            value_bin = value_bin[2:]
            while (len(value_bin) < no_qubits):
                value_bin = '0' + value_bin

            #Write in bracket notation
            value_bin = '|' + value_bin + '>'

            #Write rounded coefficient in front with plus sign behind
            value_string = str(wave_round[i]) + value_bin + ' + '

            state_string += value_string

    state_string = state_string[:-3]  #Remove the last ' + '
    return state_string


def State_to_prob(qubits):
    '''In this function we will return a list that consists of the probabilites of all the different states'''
    assert type(qubits) is Qureg

    no_qubits = len(qubits)  #number of qubits

    prob_round_list = []

    for i in range(2**no_qubits):
        value_bin = bin(i)  #returns '0b10101'
        value_bin = value_bin[2:]  #Gets rid of '0b'
        while (len(value_bin) < no_qubits):
            value_bin = '0' + value_bin
        assert len(value_bin) == no_qubits
        #because the backend.get functions see the binary numbers inverted i.e. '0001' is 8 instead of our '1000' is 8 we invert the string
        eng.flush()
        probability = eng.backend.get_probability(
            value_bin[::-1], qubits
        )  #get_amplitude only works on calling all qubits, so this scheme does not work
        prob_round = np.round(probability, 8)
        prob_round_list.append(prob_round)

    return prob_round_list

# Swap Operator and Second Renyi Entropy

We now perform a the controlled Swap operator on the first subsystem A of the qubits. We don't do the swap operator multiple times on new systems and measure the probabilities of the ancilla qubit. But we use the backend cheat function to get the probability that the ancilla qubit is in state $|0\rangle$ or in state $|1\rangle$. The second Renyi entropy is defined as 
\begin{equation}\label{eq:}
R_2 = - \log (Tr (\rho_A^2))
\end{equation}


In [ ]:
def AncillaSecondRenyiEntropy(
        qAB, qAB2, systemsizeA
):  #A function that brings the ancilla qubit in the right state and finds the second Renyi Entropy
    q_Ancilla = eng.allocate_qubit()
    
    H | q_Ancilla
    with Control(eng, q_Ancilla):
        for qubit in range(systemsizeA):
            #eng.flush()
            Swap | (qAB[qubit], qAB2[qubit])
    H | q_Ancilla  #Return the Hadamard

    eng.flush()
    prob0 = eng.backend.get_probability('0', q_Ancilla)
    prob1 = eng.backend.get_probability('1', q_Ancilla)
    
    #print(prob0)

    #Calculate Second Renyi Entropy
    R2 = prob0 - prob1
    Renyi2 = -np.log(R2) / np.log(2)

    #Bring the two systems back to the original again
    H | q_Ancilla
    with Control(eng, q_Ancilla):
        for qubit in range(systemsizeA):
            #eng.flush()
            Swap | (qAB[qubit], qAB2[qubit])
    H | q_Ancilla  
    
    #Measure and deallocate the ancilla qubit
    Measure | q_Ancilla
    del q_Ancilla

    return Renyi2

# Time Evolved Renyi Entropy

In [ ]:

#We don't use this because this need to start time evolving from the initial state all the time
def time_evolved_Renyi(timesteps, generate_superposition, Hamiltonian_trot):
    #Create the qubits of two systems, since we count the Lattice from 0 we do +1
    qureg1 = eng.allocate_qureg(N_lattice + 1)
    qureg2 = eng.allocate_qureg(N_lattice + 1)

    #Bring the two states into the desired superposition and time evolve.
    generate_superposition(qureg1)
    generate_superposition(qureg2)

    for i in range(timesteps):
        Hamiltonian_trot = trot(
    Hamiltonian_jw, evolution_time=precision_evolution_time)
        time_evolve(qureg1, Hamiltonian_trot)
        Hamiltonian_trot = trot(
    Hamiltonian_jw, evolution_time=precision_evolution_time)
        time_evolve(qureg2, Hamiltonian_trot)

    #Calculate the Renti Entropy
    Renyi2 = AncillaSecondRenyiEntropy(qureg1, qureg2, systemsizeA)

    #Deallocate the qubits
    All(Measure) | qureg1
    All(Measure) | qureg2
    del qureg1
    del qureg2

    return Renyi2

def time_evolve_one_step_Renyi(qureg1, qureg2, Hamiltonian_jw):
    #Calculate the new generator for the Hamiltonian again and time evolve one step
    #Hamiltonian_trot = trot(Hamiltonian_jw, evolution_time=precision_evolution_time)
    #print('Time to evolve the first qureg')
    time_evolve(qureg1, Hamiltonian_trot)
    #Hamiltonian_trot = trot(Hamiltonian_jw, evolution_time=precision_evolution_time)
    #print('Time to evolve the second qureg')
    time_evolve(qureg2, Hamiltonian_trot)

    #Calculate the Renyi Entropy
    #print('Time to calculate the Renyi Entropy')
    AncillaSecondRenyiEntropy(qureg1,qureg2, systemsizeA)
    Renyi2 = AncillaSecondRenyiEntropy(qureg1, qureg2, systemsizeA)

    return Renyi2

# Second Renyi Entropy Time Evolved Plot

In [ ]:
%%time
#Create the qubits of two systems, since we count the Lattice from 0 we do +1
qureg1 = eng.allocate_qureg(N_lattice)
qureg2 = eng.allocate_qureg(N_lattice)

#Set up the state |100> and time evolve
def Xfirstqubit(qureg):
    X | qureg[0]

    
#Bring the two states into the desired superposition and time evolve.
Xfirstqubit(qureg1)
Xfirstqubit(qureg2)
    
renyi2list = []

for i in range(200):
    if i%10==0: print(i, end = ' ')
    a = time_evolve_one_step_Renyi(qureg1, qureg2, Hamiltonian_jw)
    renyi2list.append(a)
    
#print(renyi2list)
    
#Measure the qubits
All(Measure) | qureg1
All(Measure) | qureg2

del qureg1
del qureg2

#print(circuit_backend.get_latex())
%matplotlib inline
ilist = []
for i in range(len(renyi2list)):
    ilist.append(i)
    
plt.plot(ilist,renyi2list)
plt.xlabel('Evolution of the Second Renyi entropy through time of the state |100>')
plt.show()

In [ ]:
fourier = np.fft.fft(renyi2list)
plt.plot(range(len(fourier)), abs(fourier))

plt.show()

In [ ]:
print(renyi2list)


In [ ]:
%matplotlib inline
ilist = []
for i in range(len(renyi2list)):
    ilist.append(i)
    
plt.plot(ilist,renyi2list)
plt.xlabel('Evolution of the Second Renyi entropy through time of the state |100>')

plt.show()

In [ ]:
import scipy.fftpack

# Number of samplepoints
N = 200
# sample spacing
T = 1.0 / 124.
x = np.linspace(0.0, N*T, N)
y = renyi2list
yf = scipy.fftpack.fft(y)
xf = np.linspace(0.0, 1.0/(2.0*T), N//2)

fig, ax = plt.subplots()
ax.plot(xf, 2.0/N * np.abs(yf[:N//2]))
plt.show()

# Visualization of the time evolved Renyi entropy

In [ ]:
#Create the qubits of two systems, since we count the Lattice from 0 we do +1
qureg1 = eng.allocate_qureg(N_lattice)

#Set up the state |100> and time evolve
def Xfirstqubit(qureg):
    X | qureg[0]
    
#Bring the state into the desired superposition and time evolve.
Xfirstqubit(qureg1)

state_string_list = []

for i in range(200):
    print(i)
    time_evolve(qureg1, Hamiltonian_trot)
    a = State_to_String(qureg1)
    state_string_list.append(a)

#Measure the qubits
All(Measure) | qureg1
del qureg1

#print(circuit_backend.get_latex())

In [ ]:
#Create the qubits of two systems, since we count the Lattice from 0 we do +1
qureg1 = eng.allocate_qureg(N_lattice)

#Set up the state |100> and time evolve
def Xfirstqubit(qureg):
    X | qureg[0]
    
#Bring the state into the desired superposition and time evolve.
Xfirstqubit(qureg1)

state_string_histprob_list = []

for i in range(200):
    print(i)
    time_evolve(qureg1, Hamiltonian_trot)
    a = State_to_prob(qureg1)
    state_string_histprob_list.append(a)

#Measure the qubits
All(Measure) | qureg1
del qureg1

#print(circuit_backend.get_latex())

In [ ]:
#Set up the animation
data = state_string_histprob_list
number_of_frames = len(state_string_histprob_list[1])
data2 = renyi2list
objects = ['000','001','010','011','100','101','110','111']

fig = plt.figure()
p1 = fig.add_subplot(2,1,1)
p2 = fig.add_subplot(2,1,2)

def update_graph(num, data, data2):
    p1.cla()
    p1.set_ylim(0.0,1.0)
    p1.bar(objects, data[num], align = 'center', alpha=0.5)
    
    p2.cla()
    p2.plot(ilist,renyi2list)
    #p2.xlim(0.0,100.0)
    #p2.ylim(0.0,1.0)
    p2.plot( ilist[num],data2[num], 'ro')

#plot = plt.plot(ilist[0],renyi2list[0], 'ro')
#plt.xlabel('Evolution of the Second Renyi entropy through time of the state |100>')

plt.cla()
ani = animation.FuncAnimation(fig, update_graph, 200, fargs=(data, data2 ) )
plt.cla()

HTML(ani.to_jshtml())    

In [ ]:
data[0]

In [ ]:
fig= plt.figure(figsize=(12,8))
p1 = fig.add_subplot(2,2,1)
p2 = fig.add_subplot(2,2,2)
p3 = fig.add_subplot(2,2,3)
p4 = fig.add_subplot(2,2,4)

fig.suptitle('Probability density function of a single fermion in the starting \n state' r' |001>'' evolved through time in the supersymmetric model.', fontsize=20)
p1.bar(objects, data[0], align = 'center', alpha=0.5)
p1.set_ylim([0,1])
p1.set_xlabel('States at time = 0')
p1.set_ylabel('Probability density of a fermion')
p2.set_ylabel('Probability density of a fermion')
p3.set_ylabel('Probability density of a fermion')
p4.set_ylabel('Probability density of a fermion')
p2.bar(objects, data[10], align = 'center', alpha=0.5)
p2.set_ylim([0,1])
p2.set_xlabel('States at time = 10')
p3.bar(objects, data[20], align = 'center', alpha=0.5)
p3.set_ylim([0,1])
p3.set_xlabel('States at time = 20')
p4.bar(objects, data[35], align = 'center', alpha=0.5)
p4.set_ylim([0,1])
p4.set_xlabel('States at time = 35')
plt.show()

In [ ]:
def XXZ_sum_terms_Pauli_random_strength(i,
                                        spin_operator=1,
                                        spin_z_operator=1,
                                        field_stregth=1):
    '''This function writes one term of the total sum of the Hamiltonian. 
        This will be used in the time evolution operator of projectq. 
        We use open boundary conditions so whenever i = (N_lattice - 1) we say (i+1) = 0 '''

    spin_operator = float(spin_operator)
    spin_z_operator = float(spin_z_operator)
    field_stregth = float(field_stregth)

    first_qubit = i

    if i == N_lattice - 1:
        second_qubit = 0

    else:
        second_qubit = i + 1

    local_strength = (
        (random.random() - 0.5) *
        2) * field_stregth  #The first term is a random number in [-1,1]

    term_X = QubitOperator(f'X{first_qubit} X{second_qubit}', spin_operator)
    term_Y = QubitOperator(f'Y{first_qubit} Y{second_qubit}', spin_operator)
    term_Z = QubitOperator(f'Z{first_qubit} Z{second_qubit}', spin_z_operator)
    local_term = QubitOperator(f'Z{i}', local_strength)

    term_sum = term_X + term_Y + term_Z + local_term

    return term_sum


def XXZ_Hamiltonian(spin_operator=1, spin_z_operator=1, field_stregth=1):

    ham = QubitOperator()  #Empty qubit operator

    for i in range(N_lattice):
        term = XXZ_sum_terms_Pauli_random_strength(
            i, spin_operator, spin_z_operator, field_stregth)
        ham += term

    assert type(ham) == QubitOperator

    return ham

In [ ]:
data = state_string_histprob_list_XXZ
objects = ['000','001','010','011','100','101','110','111']

fig= plt.figure(figsize=(12,8))
p1 = fig.add_subplot(2,2,1)
p2 = fig.add_subplot(2,2,2)
p3 = fig.add_subplot(2,2,3)
p4 = fig.add_subplot(2,2,4)

fig.suptitle('Probability density function of a single fermion in the starting \n state' r' |001>'r' evolved through time in the XXZ with J = 1 = $\Delta$, h = 1 model.', fontsize=20)
p1.bar(objects, data[0], align = 'center', alpha=0.5)
p1.set_ylim([0,1])
p1.set_xlabel('States at time = 0')
p1.set_ylabel('Probability density of a fermion')
p2.set_ylabel('Probability density of a fermion')
p3.set_ylabel('Probability density of a fermion')
p4.set_ylabel('Probability density of a fermion')
p2.bar(objects, data[10], align = 'center', alpha=0.5)
p2.set_ylim([0,1])
p2.set_xlabel('States at time = 10')
p3.bar(objects, data[20], align = 'center', alpha=0.5)
p3.set_ylim([0,1])
p3.set_xlabel('States at time = 20')
p4.bar(objects, data[35], align = 'center', alpha=0.5)
p4.set_ylim([0,1])
p4.set_xlabel('States at time = 35')
plt.show()

In [ ]:
XXZ_Hamiltonian(1,1,1)

In [ ]:
import random

#Create the qubits of two systems, since we count the Lattice from 0 we do +1
qureg1 = eng.allocate_qureg(N_lattice)

#Set up the state |100> and time evolve
def Xfirstqubit(qureg):
    X | qureg[0]
    
#Bring the state into the desired superposition and time evolve.
Xfirstqubit(qureg1)
ham_XXZ = XXZ_Hamiltonian(1, 1, 1)
print(ham_XXZ)
ham_trot = trot(ham_XXZ)
print(list(ham_trot)) 

state_string_histprob_list_XXZ = []

for i in range(200):
    print(i)
    a = State_to_prob(qureg1)
    state_string_histprob_list_XXZ.append(a)
    time_evolve(qureg1, list(trot(ham_XXZ)))


#Measure the qubits
All(Measure) | qureg1
del qureg1

#print(circuit_backend.get_latex())

In [ ]:
ilist = []
for i in range(len(renyi2list)):
    ilist.append(i)

#Set up the animation
data = state_string_histprob_list
number_of_frames = len(state_string_histprob_list[1])
data2 = renyi2list

fig = plt.figure()
p1 = fig.add_subplot(2,1,1)
p2 = fig.add_subplot(2,1,2)

def update_graph(num, data, data2):
    p1.cla()
    p1.set_ylim(0.0,1.0)
    p1.bar(objects, data[num], align = 'center', alpha=0.5)
    
    p2.cla()
    p2.plot(ilist,renyi2list)
    #p2.xlim(0.0,100.0)
    #p2.ylim(0.0,1.0)
    p2.plot( ilist[num],data2[num], 'ro')

#plot = plt.plot(ilist[0],renyi2list[0], 'ro')
#plt.xlabel('Evolution of the Second Renyi entropy through time of the state |100>')

plt.cla()
ani = animation.FuncAnimation(fig, update_graph, 100, fargs=(data, data2 ) )
plt.cla()

HTML(ani.to_jshtml())    
ani.save('the_movie.mp4', writer = 'mencoder', fps=15)